<a href="https://colab.research.google.com/github/tavjo/computer-vision-project/blob/main/SNAI_challenge_YOLO_colab_TVJ_run2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2024 Coding Challenge: Automated Object Detection and Counting**
**Author:** Taïsha V. Joseph

**Objective:** Develop a Python-based application that uses computer vision techniques to count and classify different objects from a set of images. The system should be able to differentiate at least three types of objects (e.g., cars, bicycles, pedestrians) in diverse lighting and background conditions.

**Resources:**
- http://cocodataset.org/
- https://github.com/cocodataset/cocoapi/tree/master
- https://github.com/ultralytics/yolov5
- https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#clearml-logging-and-automation-new
- https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb

## Download Data

- Get COCO training and validation datasets
- Locally, the zip files can be downloaded and extracted using the Python script *download_data.py*
- In the Google Colab notebook, for faster download, the zip files were stored in my Google Drive and anyone with the links can download the datasets to Google Colab using *gdrive*.
- The code for downloading the datasets from Google Drive or directly from the COCO website can be found in the cells below.

In [7]:
!rm -r /content/sample_data

rm: cannot remove '/content/sample_data': No such file or directory


In [8]:
# Do not change this code
!pip install --upgrade --no-cache-dir gdown

Organize local directory before getting datasets

In [9]:
!mkdir /content/data/
!mkdir /content/data/filtered/
!mkdir /content/data/tmp/

mkdir: cannot create directory ‘/content/data/’: File exists
mkdir: cannot create directory ‘/content/data/filtered/’: File exists
mkdir: cannot create directory ‘/content/data/tmp/’: File exists


In [10]:
# Helper functions
def generate_direct_download_link(shared_link):
    """
    Generates a direct download link from a shared Google Drive link.

    Args:
        shared_link (str): The shared Google Drive link.

    Returns:
        str: The direct download link or an error message.

    Raises:
        ValueError: If the provided link is not a valid Google Drive shared link.
    """
    try:
        # Check if the shared link is a valid Google Drive link
        if "drive.google.com/file/d/" not in shared_link:
            raise ValueError("The provided link is not a valid Google Drive shared link.")

        # Extract the file ID from the shared link
        file_id = shared_link.split('/d/')[1].split('/')[0]

        # Create the direct download link
        direct_download_link = f"https://drive.google.com/uc?export=download&id={file_id}"
        print (direct_download_link)

        return direct_download_link
    except Exception as e:
        return f"An error occurred: {e}"


In [11]:
# Google Drive shared link for the necessary files
val2017 = "https://drive.google.com/file/d/18rji_6JgJYIBfv7EY2PQP3nQA689ZR2I/view?usp=sharing"
train2017 = "https://drive.google.com/file/d/1RgUrTkclcqtT8FVl912XCzX3jIahQwBU/view?usp=sharing"
annotations_trainval2017 = "https://drive.google.com/file/d/10oTb3FBJbWz7ZKjw2G-udamNMCBfYtuF/view?usp=sharing"

In [12]:
# transform shared google drive shared links into direct download links
train2017 = generate_direct_download_link(train2017)
val2017 = generate_direct_download_link(val2017)
annotations_trainval2017 = generate_direct_download_link(annotations_trainval2017)

https://drive.google.com/uc?export=download&id=1RgUrTkclcqtT8FVl912XCzX3jIahQwBU
https://drive.google.com/uc?export=download&id=18rji_6JgJYIBfv7EY2PQP3nQA689ZR2I
https://drive.google.com/uc?export=download&id=10oTb3FBJbWz7ZKjw2G-udamNMCBfYtuF


In [13]:
#export zip data files from google drive
!gdown "$train2017" -O /content/data/tmp/train2017.zip
!gdown "$val2017" -O /content/data/tmp/val2017.zip
!gdown "$annotations_trainval2017" -O /content/data/tmp/annotations_trainval2017.zip

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?export=download&id=1RgUrTkclcqtT8FVl912XCzX3jIahQwBU

but Gdown can't. Please check connections and permissions.
Downloading...
From (original): https://drive.google.com/uc?export=download&id=18rji_6JgJYIBfv7EY2PQP3nQA689ZR2I
From (redirected): https://drive.google.com/uc?export=download&id=18rji_6JgJYIBfv7EY2PQP3nQA689ZR2I&confirm=t&uuid=19deee02-6e73-4c8f-84b0-79fce78af2a3
To: /content/data/tmp/val2017.zip
100% 816M/816M [00:17<00:00, 47.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?export=download&i

**Optional:** Download COCO datasets directly from the COCO website (takes longer than exporting from Google Drive)

In [17]:
import torch
# alternate download method for COCO datasets
#since temporarily, I can't download the train dataset from my google drive, will provide alternative method

# COCO dataset URLs (example URLs, modify as needed)
# train_url = "http://images.cocodataset.org/zips/train2017.zip"
# val_dataset_url = "http://images.cocodataset.org/zips/val2017.zip"
# annotation_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

# import torch
# torch.hub.download_url_to_file(train_url, '/content/data/tmp/train2017.zip')
# torch.hub.download_url_to_file(val_dataset_url, '/content/data/tmp/val2017.zip')
# torch.hub.download_url_to_file(annotation_url, '/content/data/tmp/annotations_trainval2017.zip')


100%|██████████| 18.0G/18.0G [16:35<00:00, 19.4MB/s]


In [18]:
# Extract Data from zipped files
import os
import zipfile

local_zip = '/content/data/tmp/train2017.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/data/tmp')
local_zip = '/content/data/tmp/val2017.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/data/tmp')
local_zip = '/content/data/tmp/annotations_trainval2017.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/data/tmp')
zip_ref.close()

## 2.Data Prep

2.1. Filter COCO Annotations
- Filter the annotations for the classes person, car, and bicycle.
- In this section, I will update the annotations and copy the relevant images to a new directory.
- *Load the COCO annotations using pycocotools (input_annotation_file).*

    - Get category IDs for the classes of interest using getCatIds(catNms=categories_to_keep).
    - Get image IDs associated with the filtered categories using getImgIds(catIds=category_ids).

- *Filtering Process:*

    - Iterate over the filtered image IDs.
    - Load image information and corresponding annotations.
    - Append filtered images and annotations to the respective lists.
    - Copy relevant images to the output directory.
    
- *Saving Filtered Data:*

    - Save the filtered images, annotations, and categories to the output annotation file in COCO format.

**Counting and Sampling:** The code counts the number of images per class and samples an equal number of images from each class to balance the dataset.
**Incorporating Non-specific Images:** It randomly selects a specified number of images that do not contain any of the target classes and adds them to the final dataset.

In [19]:
!pip install pycocotools

In [20]:
from pycocotools.coco import COCO
import os
import shutil
import json
import random

def ensure_dir_exists(directory):
    """
    Ensure that a directory exists. If it does not, create it.

    Parameters:
    directory (str): The directory path to check/create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

def filter_coco_annotations(input_annotation_file, output_annotation_file, image_dir, output_image_dir, categories_to_keep, num_non_specific_images):
    """
    Filter COCO annotations to only include specified categories and add non-specific images.

    Parameters:
    input_annotation_file (str): Path to the input COCO annotations file.
    output_annotation_file (str): Path to the output filtered annotations file.
    image_dir (str): Directory containing the input images.
    output_image_dir (str): Directory to store the filtered images.
    categories_to_keep (list): List of category names to keep.
    num_non_specific_images (int): Number of non-specific images to include.

    Raises:
    ValueError: If the number of non-specific images requested exceeds the available non-specific images.
    """
    try:
        # Load COCO dataset
        coco = COCO(input_annotation_file)
        category_ids = coco.getCatIds(catNms=categories_to_keep)
        image_ids_per_category = {cat_id: coco.getImgIds(catIds=[cat_id]) for cat_id in category_ids}

        # Determine the minimum number of images per category
        min_images_per_category = min(len(ids) for ids in image_ids_per_category.values())
        print(f"Minimum images per category: {min_images_per_category}")

        # Sample min_images_per_category images per category
        balanced_image_ids = []
        for cat_id, img_ids in image_ids_per_category.items():
            balanced_image_ids.extend(random.sample(img_ids, min_images_per_category))

        # Collect all non-specific images
        all_image_ids = set(coco.getImgIds())
        specific_image_ids = set(balanced_image_ids)
        non_specific_image_ids = list(all_image_ids - specific_image_ids)

        # Ensure there are enough non-specific images
        if num_non_specific_images > len(non_specific_image_ids):
            raise ValueError("Requested number of non-specific images exceeds available non-specific images.")

        # Sample non-specific images
        sampled_non_specific_image_ids = random.sample(non_specific_image_ids, num_non_specific_images)

        # Combine balanced specific and sampled non-specific images
        final_image_ids = balanced_image_ids + sampled_non_specific_image_ids

        filtered_annotations = []
        filtered_images = []

        # Ensure output directories exist
        ensure_dir_exists(output_image_dir)
        ensure_dir_exists(os.path.dirname(output_annotation_file))

        # Process each image
        for img_id in final_image_ids:
            img_info = coco.loadImgs(img_id)[0]
            ann_ids = coco.getAnnIds(imgIds=img_id, catIds=category_ids if img_id in balanced_image_ids else [])
            anns = coco.loadAnns(ann_ids)

            filtered_images.append(img_info)
            filtered_annotations.extend(anns)

            # Copy image to output directory
            src_img_path = os.path.join(image_dir, img_info['file_name'])
            dst_img_path = os.path.join(output_image_dir, img_info['file_name'])
            shutil.copy(src_img_path, dst_img_path)

        # Create filtered data dictionary
        filtered_data = {
            'images': filtered_images,
            'annotations': filtered_annotations,
            'categories': [cat for cat in coco.loadCats(category_ids)]
        }

        # Write filtered data to JSON file
        with open(output_annotation_file, 'w') as f:
            json.dump(filtered_data, f)

        print("Filtering and copying completed successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [21]:
categories_to_keep = ['person', 'car', 'bicycle']

In [31]:
filter_coco_annotations(
    '/content/data/tmp/annotations/instances_train2017.json',
    '/content/data/filtered/annotations/filtered_instances_train2017.json',
    '/content/data/tmp/train2017',
    '/content/data/filtered/images/train2017',
    categories_to_keep,
    0
)

loading annotations into memory...
Done (t=14.74s)
creating index...
index created!
Minimum images per category: 3252
Filtering and copying completed successfully.


In [32]:
filter_coco_annotations(
    '/content/data/tmp/annotations/instances_val2017.json',
    '/content/data/filtered/annotations/filtered_instances_val2017.json',
    '/content/data/tmp/val2017',
    '/content/data/filtered/images/val2017',
    categories_to_keep,
    0
)

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!
Minimum images per category: 149
Filtering and copying completed successfully.


In [ ]:
# !rm -r /content/data/tmp

2.2 Convert COCO Annotations to YOLO Format
Convert the filtered COCO annotations to YOLO format (i.e., .txt files as labels for each image).

In [34]:

def convert_coco_to_yolo(annotations_file, labels_dir, categories_to_keep):
    """
    Convert COCO annotations to YOLO format using pycocotools.

    Parameters:
    annotations_file (str): Path to the COCO annotations JSON file.
    labels_dir (str): Directory to save the YOLO format label files.
    categories_to_keep (list): List of category names to keep.

    Raises:
    FileNotFoundError: If the annotations file does not exist.
    KeyError: If required keys are missing in the annotations file.
    """
    try:
        # Load COCO dataset
        if not os.path.exists(annotations_file):
            raise FileNotFoundError(f"Annotations file {annotations_file} does not exist.")

        coco = COCO(annotations_file)

        # Get category IDs and category names
        category_ids = coco.getCatIds(catNms=categories_to_keep)
        categories = coco.loadCats(category_ids)
        categories_dict = {cat['id']: cat['name'] for cat in categories}

        # Create a dictionary to map category names to YOLO class indices
        category_to_index = {name: index for index, name in enumerate(categories_dict.values())}

        # Ensure the labels directory exists
        os.makedirs(labels_dir, exist_ok=True)

        # Create a set of all images containing the categories of interest
        image_ids_of_interest = set(coco.getImgIds(catIds=category_ids))

        # Iterate over all images
        for img in coco.loadImgs(coco.getImgIds()):
            image_id = img['id']
            image_info = coco.loadImgs(image_id)[0]
            image_width = image_info['width']
            image_height = image_info['height']

            # Initialize the label file path
            label_file_path = os.path.join(labels_dir, f"{image_info['file_name'].split('.')[0]}.txt")

            # Check if the image contains any of the categories of interest
            if image_id in image_ids_of_interest:
                # Get annotations for the image
                ann_ids = coco.getAnnIds(imgIds=image_id, catIds=category_ids)
                anns = coco.loadAnns(ann_ids)

                # Write YOLO format labels for the image
                with open(label_file_path, 'w') as label_file:
                    for ann in anns:
                        category_id = ann['category_id']
                        bbox = ann['bbox']

                        # Get the category name and check if it's in the category_to_index dictionary
                        category_name = categories_dict.get(category_id)
                        if category_name not in category_to_index:
                            continue

                        # Calculate YOLO format coordinates (normalized)
                        x_center = (bbox[0] + bbox[2] / 2) / image_width
                        y_center = (bbox[1] + bbox[3] / 2) / image_height
                        width = bbox[2] / image_width
                        height = bbox[3] / image_height

                        # Create the YOLO label string
                        yolo_label = f"{category_to_index[category_name]} {x_center} {y_center} {width} {height}\n"

                        # Write the label to the file
                        label_file.write(yolo_label)
            # else:
            #     # Create an empty label file for non-specific images
            #     with open(label_file_path, 'w') as label_file:
            #         pass

        print("Conversion to YOLO format completed successfully.")

    except FileNotFoundError as fnf_error:
        print(f"File not found: {fnf_error}")
    except KeyError as key_error:
        print(f"Key error: {key_error}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [35]:
convert_coco_to_yolo(
    '/content/data/filtered/annotations/filtered_instances_train2017.json',
    '/content/data/filtered/labels/train2017',
    categories_to_keep
)

loading annotations into memory...
Done (t=0.89s)
creating index...
index created!
Conversion to YOLO format completed successfully.


In [36]:
convert_coco_to_yolo(
    '/content/data/filtered/annotations/filtered_instances_val2017.json',
    '/content/data/filtered/labels/val2017',
    categories_to_keep
)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Conversion to YOLO format completed successfully.


## **Train**
**2 Options for training a model with YOLOv5:**

- Fine-tune a pre-trained model with pre-exisiting weights
- **Train a model from scratch**


## **yolov5n architecture:**
### Loss functions:
Classification loss:
  1. Binary Cross-Entropy with Logits Loss (BCEWithLogitsLoss)
  2. FocalLoss (variant of cross-entropy that handles class imbalance) if gamma (focusing param) > 0
Bounding Box regression loss:
  1. Complete Intersection over Union (IoU) loss

### Activation function
- SiLU (Swish) Activation Function (default)

### Hyperparams:

```yaml
# Ultralytics YOLOv5 🚀, AGPL-3.0 license (customized)

# Parameters
nc: 3 # number of classes (modified)
depth_multiple: 0.5 # model depth multiple (modified)
width_multiple: 0.5 # layer channel multiple (modified)
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, nn.Dropout, [0.2]],  # Adding Dropout with 20% probability
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13
    [-1, 1, nn.Dropout, [0.2]],  # Adding Dropout with 20% probability

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)
    [-1, 1, nn.Dropout, [0.2]],  # Adding Dropout with 20% probability

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)
    [-1, 1, nn.Dropout, [0.3]],  # Adding Dropout with 30% probability

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)
    [-1, 1, nn.Dropout, [0.5]],  # Adding Dropout with 50% probability

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]
```

### Train Model (smallest one is yolov5n) from scratch
Should be in *~/Documents/Projects/yolov5/* directory

- Navigate to the models folder in the cloned yolov5 repo
- Modify the "yolov5n.yaml" file by replacing the number of classes from 80 to 3
- Adjust other parameters for each layer of the model as needed. (Let's start with the default parameters)

**Define parameters for training:**

- img 640: Use 640x640 images for training.
- batch 16: Use a batch size of 16.
- epochs 50: Train for 200 epochs.
- data coco_person_car_bicycle.yaml: Use custom data configuration file with fewer classes.
- weights '': Train from scratch by not using any pre-trained weights.
- cfg yolov5n.yaml: Use the configuration file for YOLOv5n (the smallest model). --> "models/yolov5n.yaml"
--project:Directory where results will be saved.

In [27]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16634, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 16634 (delta 70), reused 105 (delta 47), pack-reused 16459
Receiving objects: 100% (16634/16634), 15.29 MiB | 11.68 MiB/s, done.
Resolving deltas: 100% (11376/11376), done.


In [20]:
# %pip install -r /content/yolov5/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.8/782.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


**Upload data configuration file and place in main directory of yolov5 repo.**
- It should be a .yaml file with the following contents:

```yaml
  # coco_person_car_bicycle.yaml
  # If local
  # train: /Users/taishajoseph/Documents/Projects/computer-vision-project/data/filtered/images/train2017/
  # val: /Users/taishajoseph/Documents/Projects/computer-vision-project/data/filtered/images/val2017/
  # If in Google Colab
  train: /content/data/filtered/images/train2017/
  val: /content/data/filtered/images/val2017/
  test:  # Optional

  # number of classes
  nc: 3

  # class names
  names: ['person', 'car', 'bicycle']
  ```

In [ ]:
# !cp /content/coco_person_car_bicycle.yaml /content/yolov5/coco_person_car_bicycle.yaml

cp: cannot stat '/content/coco_person_car_bicycle.yaml': No such file or directory


**Choose a model from the yolov5 models directory**
- In this notebook, we will use yolov5n, the smallest model
- Modify the contents of the model configuration file to include the correct number of classes
- Make any other changes as

In [28]:
%cd yolov5

/content/yolov5


**Identify optimal anchor boxes**

In [37]:
# !python train.py --img 640 --batch 16 --epochs 10 --data coco_person_car_bicycle.yaml --cfg yolov5n_mod.yaml --weights '' --nosave --cache

2024-06-17 18:35:59.385887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 18:35:59.385940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 18:35:59.387334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=yolov5n_mod.yaml, data=coco_person_car_bicycle.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, 

**Replace default anchors in yolov5n_mod.yaml with optimal anchor boxes to improve model performance.**

In [38]:
!python train.py --img 640 --batch 16 --epochs 200 --data coco_person_car_bicycle.yaml --weights '' --cfg yolov5n_mod.yaml --cache

2024-06-17 18:38:29.156856: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 18:38:29.156911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 18:38:29.158495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=yolov5n_mod.yaml, data=coco_person_car_bicycle.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False

### Visualize Results and Examine Performance Metrics

In [1]:
# import matplotlib.pyplot as plt
from IPython.display import Image, display

# Open the image file
display(Image(filename="runs/train/exp3/results.png"))

FileNotFoundError: [Errno 2] No such file or directory: 'runs/train/exp3/results.png'

### **Performance Metrics Summary**

1. **train/box_loss**: Indicates model improvement in predicting bounding boxes as training progresses, shown by decreasing loss.
2. **train/obj_loss**: Shows improvement in detecting objects and distinguishing them from the background, indicated by a decreasing trend.
3. **train/cls_loss**: Reflects better performance in classifying detected objects, indicated by decreasing loss.
4. **metrics/precision**: Increasing precision shows a reduction in false positives, indicating improved model performance.
5. **metrics/recall**: Increasing recall indicates the model is identifying more true positives, though a recall above 70% is preferable.
6. **val/box_loss**: Decreasing validation box loss shows better generalization to new data.
7. **val/obj_loss**: A decreasing trend indicates improved object detection in new data.
8. **val/cls_loss**: Decreasing validation classification loss indicates better object classification in new data.
9. **metrics/mAP_0.5**: Increasing mAP_0.5 indicates better precision-recall trade-off at an IoU threshold of 0.5.
10. **metrics/mAP_0.5:0.95**: Increasing mAP_0.5:0.95 indicates good model performance across various IoU thresholds.

#### **Conclusion and Suggestions for Improvement**

**Conclusion**: The model is performing well, with consistent improvements in both training and validation phases. Decreasing losses and increasing metrics for precision, recall, and mAP indicate effective learning and generalization capabilities.

**Suggestions for Improvement**:
- Increasing the number of training epochs from 50 to 200 would further improve model performance.
- Increase depth and width of network
- Add more layers
- More complex data augmentation methods


### Export results

In [ ]:
import shutil
from google.colab import files

# Step 1: Zip the folder
folder_to_zip = 'runs/train/exp3'  # Replace with your folder name
output_filename = '/Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/train/run2.zip'  # Replace with your desired zip file name

shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)

# Step 2: Download the zipped folder
files.download(output_filename)


## Evaluate

In [ ]:
!python val.py --data "coco_person_car_bicycle.yaml" --weights "runs/train/exp3/weights/best.pt"

### Visualize Results

In [ ]:
# import matplotlib.pyplot as plt
# from IPython.display import Image, display

# Open the image file
display(Image(filename="runs/val/exp/confusion_matrix.png"))

# Run 1:

**Summary of Initial Model Performance on Validation data:**
- Person detection: correctly identifies person 30% of the time and misclassifies a person 69% of the time
- Car detection: correctly identifies car 26% of the time and misclassifies a car 74% of the time
- Bicycle detection:correctly identifies car 32% of the time and misclassifies a car 68% of the time

**How to Improve Classification?**
- The performance for "person," "car," and "bicycle" needs improvement. The high rates of misclassification, especially with the background, indicate that the model may benefit from additional training data, better feature extraction, or tuning of hyperparameters.
- If the dataset is imbalanced, techniques such as resampling, data augmentation, or using class weights can be utilized address the imbalance.

# Run 2:

**Summary of Model Performance on Validation data:**
- Person detection: correctly identifies person % of the time and misclassifies a person % of the time
- Car detection: correctly identifies car % of the time and misclassifies a car % of the time
- Bicycle detection:correctly identifies car % of the time and misclassifies a car % of the time

In [ ]:
# import shutil
# from google.colab import files

# Step 1: Zip the folder
folder_to_zip = 'runs/val/exp'  # Replace with your folder name
output_filename = '/Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/val/val2.zip'  # Replace with your desired zip file name

shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)

# Step 2: Download the zipped folder
files.download(output_filename)

## Inference

In [ ]:
!python detect.py --weights "runs/train/exp3/weights/best.pt" --img 640 --conf 0.25 --source data/images

In [ ]:
display(Image(filename='runs/detect/exp/bus.jpg', width=600))

In [ ]:
display(Image(filename='runs/detect/exp/zidane.jpg', width=600))

## Clean Up

Terminate the kernel and free memory:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

#### References
- Jocher, G. (2020). YOLOv5 by Ultralytics (Version 7.0) [Computer software]. https://doi.org/10.5281/zenodo.3908559
- https://medium.com/@henriquevedoveli/metrics-matter-a-deep-dive-into-object-detection-evaluation-ef01385ec62
- https://vignesh943628.medium.com/metrics-on-object-detection-b9fe3f1bac59
- https://towardsdatascience.com/map-mean-average-precision-might-confuse-you-5956f1bfa9e2